In [ ]:
from pathlib import Path
import rasterio
import geopandas as gpd
import data as dt

data_dir = Path("data/raw")
img = rasterio.open(Path(data_dir) / "GL085796E28265N.tif")
y = gpd.read_file(data_dir / "GL_3basins_2015.shp")
result = dt.preprocessor(img, y)

In [ ]:
import matplotlib.pyplot as plt
for s in result[2]:
    plt.imshow(s)
    plt.show()

In [ ]:
np.save(data_dir / "y.npy", result[2][0])
np.save(data_dir / "exhm.npy", result[2][1])
np.save(data_dir / "dts.npy", result[2][2])
np.save(data_dir / "sdts.npy", result[2][3])
np.save(data_dir / "x.npy", img.read())